In [1]:
import requests
import urllib.request
from urllib.parse import quote
import numpy as np
from tqdm import tqdm
import contextlib
import json
import tqdm
from bs4 import BeautifulSoup as bs
import time
import matplotlib.pyplot as plt
import collections
import pandas as pd

### Set up spotify scrapping functions

In [2]:
token = 'BQBlWwBaYmPf9rBKfbSYad_9fMzogOuL1jFcGMk5vl4yC7UXc1EGE1RwN_nlYlRmhNJUt61M0dqytfMjxxcqGPQUbbOr4UCs78xgEbg0g7pzlvd68JCyJcvJL4Puy-Wg6JQveBvnKYA_NQ'

header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

final_json = []

In [3]:
def album_spotify_id(album_name, artist_name=None):
    query = None
    if artist_name is None:
        query = quote(album_name) + '&type=album'
    else:
        query = 'album:' + quote(album_name) + '%20artist:' + quote(artist_name) + '&type=album'
    r = requests.request('get', 'https://api.spotify.com/v1/search?q=' + query, headers=header)
    
    if len(r.json()['albums']['items']) != 0:
        return r.json()['albums']['items'][0]['id']
    else:
        return None

def album_tracklist(album_id):
    query = 'albums/' + album_id + '/tracks'
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return [(song['id'], song['name']) for song in r.json()['items']]

def song_features(song_id):
    query = 'audio-features/' + str(song_id)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()

def song_features_for_album(album_name, artist_name=None):
    album_id = album_spotify_id(album_name, artist_name)
    if album_id is None:
        return None
    tracklist = album_tracklist(album_id)        
    ids = [song[0] for song in tracklist]
    names = [song[1] for song in tracklist]
    return song_features(ids, names)

def get_rateYourMusic_songs(genre, filenames, table_index=3) :
    
    #song year, artist and title
    searches = []
    all_info = []
    
    for f in filenames :

        res = bs(open(f),"html5lib")
        for t in res.findAll(attrs={'class': "mbgen"})[table_index].findAll("tr")[1:] :
                        
            if len(t.findAll("td")) >= 3 :
                
                
            
                year = t.findAll("td")[0].text
                artist = t.findAll("td")[1].text
                title = t.findAll("td")[2].text

                if '/' in title :
                    searches.append([year, artist, title.split('/')[0].strip()])
                    searches.append([year, artist, title.split('/')[1].strip()])

                else :
                    searches.append([year, artist, title])

    found = 0

    for s in tqdm.tqdm(searches) :
        result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(s[2]) + "%20artist:" + quote(s[1]) + "&type=track", headers=header).json()
        time.sleep(0.2)
        items = result['tracks']['items']


        #We found the song
        if len(items) != 0 :

            found += 1

            song_id = result['tracks']['items'][0]['id']   
            song_release = items[0]['album']['release_date']
            features = song_features(song_id)

            year = s[0]
            cover = ""
            
            if len(items[0]['album']['images']) != 0 :                
                cover = items[0]['album']['images'][0]['url']            
            else :
                cover = "https://vignette.wikia.nocookie.net/uncyclopedia/images/a/a0/Questionmark.jpg/revision/latest?cb=20090616065802&format=original"
            
            artists = s[1]
            name = s[2]
            album = items[0]['album']['name']



            song_dict = {'year' : year,
                         'artists' : artists,
                         'name' : name,
                         'cover' : cover,
                         'album' : album,
                         'genre' : genre,
                        'danceability' : features['danceability'],
                        'energy' : features['energy'],
                        'key' : features['key'],
                        'loudness' : features['loudness'],
                        'mode' : features['mode'],
                        'speechiness' : features['speechiness'],
                        'acousticness' : features['acousticness'],
                        'instrumentalness' : features['instrumentalness'],
                        'liveness' : features['liveness'],
                        'valence' : features['liveness'],
                        'tempo' : features['tempo'],
                        'duration_ms' : features['duration_ms']}


            all_info.append(song_dict)
    print("found : ", found , ", (" + str(100*found/len(searches)) + "%) for genre " + genre)
    return all_info


# Download Contemporary RnB titles (94 songs)

From Itunes (70)

In [4]:
# get names from itunes
url = 'https://itunes.apple.com/search?term=song&genreId=1136&limit=200'

response = requests.get(url)
data = response.json()

found = 0

for e in tqdm.tqdm(data['results']):
    result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(e['trackName']) + "%20artist:" + quote(e['artistName']) + "&type=track", headers=header).json()
    
    items = result['tracks']['items']
    
    
    #We found the song
    if len(items) != 0 :
        
        found += 1
    
        song_id = result['tracks']['items'][0]['id']    
        features = song_features(song_id)

        year = e['releaseDate'][:4]
        cover = e['artworkUrl100']
        artists = [a['name'] for a in result['tracks']['items'][0]['artists']]
        name = e['trackName']
        album = result['tracks']['items'][0]['album']['name']



        song_dict = {'year' : year,
                     'artists' : artists,
                     'name' : name,
                     'cover' : cover,
                     'album' : album,
                     'genre' : "contemporary-rnb",
                    'danceability' : features['danceability'],
                    'energy' : features['energy'],
                    'key' : features['key'],
                    'loudness' : features['loudness'],
                    'mode' : features['mode'],
                    'speechiness' : features['speechiness'],
                    'acousticness' : features['acousticness'],
                    'instrumentalness' : features['instrumentalness'],
                    'liveness' : features['liveness'],
                    'valence' : features['liveness'],
                    'tempo' : features['tempo'],
                    'duration_ms' : features['duration_ms']}


        final_json.append(song_dict)

100%|██████████| 95/95 [00:16<00:00,  5.26it/s]


From rateYourMusic (24 songs)

In [5]:
final_json.extend(get_rateYourMusic_songs("contemporary-rnb", ["data/contemporary_rnb/cont_songs_1.html"], table_index=2))

100%|██████████| 40/40 [00:14<00:00,  2.77it/s]

found :  24 , (60.0%) for genre contemporary-rnb


# Download british RnB (223 songs)

In [6]:
final_json.extend(get_rateYourMusic_songs("british-rnb", ["data/british_rnb/brit_songs_" + str(c) + ".html" for c in range(1, 11) ]))

100%|██████████| 356/356 [02:17<00:00,  2.56it/s]

found :  223 , (62.640449438202246%) for genre british-rnb


# Download alternative RnB (198 songs)

In [7]:
final_json.extend(get_rateYourMusic_songs("alternative-rnb", ["data/alternative_rnb/alt_songs_" + str(c) + ".html" for c in range(1, 12) ]))

100%|██████████| 278/278 [02:16<00:00,  2.67it/s]

found :  198 , (71.22302158273381%) for genre alternative-rnb


# Download new jack swing RnB (197 songs)

In [8]:
final_json.extend(get_rateYourMusic_songs("new-jack-swing-rnb", ["data/new_jack_swing_rnb/njs_rnb_" + str(c) + ".html" for c in range(1, 12) ]))

100%|██████████| 300/300 [01:55<00:00,  2.81it/s]

found :  197 , (65.66666666666667%) for genre new-jack-swing-rnb


# Download new Orleans RnB (253 songs)

In [9]:
final_json.extend(get_rateYourMusic_songs("new-orleans-rnb", ["data/new_orleans_rnb/new_orleans_rnb" + str(c) + ".html" for c in range(1, 12) ]))

100%|██████████| 394/394 [02:35<00:00,  2.95it/s]

found :  253 , (64.21319796954315%) for genre new-orleans-rnb


# Download Jump Blues (261 songs)

In [10]:
final_json.extend(get_rateYourMusic_songs("jump-blues", ["data/jump_blues_rnb/jump_blues_" + str(c) + ".html" for c in range(1, 12) ]))

100%|██████████| 414/414 [02:43<00:00,  2.83it/s]

found :  261 , (63.04347826086956%) for genre jump-blues


# Download blue-eyed soul (275 songs)

In [ ]:
final_json.extend(get_rateYourMusic_songs("blue-eyed-soul", ["data/blue_eyed_soul/blue_eyed_soul_" + str(c) + ".html" for c in range(1, 12) ]))

 82%|████████▏ | 316/386 [02:08<00:29,  2.38it/s]

In [ ]:
#Dump everything into the data
json.dump(final_json, open("rnb_data.json", 'w'))

## Bar plot of nb of songs per years per genre

In [ ]:
plt.figure(figsize=(20,10))

to_plot = np.array([[d['year'][:3], d['genre']] for d in final_json if len(d['year']) == 4])
genres = to_plot[:,1]
years = to_plot[:,0]
years

pd.Series(years).value_counts().sort_index().plot(kind='bar')